In [1]:
# Import dependencies
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime
import datetime as dt
import time
import requests
import json
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import locale
locale.setlocale(locale.LC_ALL,'')

from config import apikey

In [2]:
# Using Alpha Vantange API - Find data for the SPY ticker

data_frequency = "MONTHLY"
SPY_ticker = "SPY"

SPY_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{data_frequency}&symbol={SPY_ticker}&apikey=' + apikey
SPY_response = requests.get(SPY_url).json()

In [3]:
SPY_symbol = SPY_response["Meta Data"]["2. Symbol"]

print(SPY_symbol)

SPY


In [4]:
SPY_dates = []
SPY_close = []

for dates in SPY_response["Monthly Time Series"]:
        SPY_close.append(SPY_response["Monthly Time Series"][dates]["4. close"])
        if dates not in SPY_dates:
            SPY_dates.append(dates)


SPY_data = pd.DataFrame({"Dates": SPY_dates, "SPY_Close": SPY_close})
SPY_data            

,Dates,SPY_Close
0,2020-01-03,322.4100
1,2019-12-31,321.8600
2,2019-11-29,314.3100
3,2019-10-31,303.3300
4,2019-09-30,296.7700
...,...,...
235,2000-06-30,145.2812
236,2000-05-31,142.8125
237,2000-04-28,145.0937
238,2000-03-31,150.3750


In [5]:
# Using Alpha Vantange API - Find data for the VBMFX ticker

VBMFX_ticker = "VBMFX"

VBMFX_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{data_frequency}&symbol={VBMFX_ticker}&apikey=' + apikey
VBMFX_response = requests.get(VBMFX_url).json()

VBMFX_symbol = VBMFX_response["Meta Data"]["2. Symbol"]

print(VBMFX_symbol)

VBMFX


In [6]:
VBMFX_dates = []
VBMFX_close = []

for dates in VBMFX_response["Monthly Time Series"]:
        VBMFX_close.append(VBMFX_response["Monthly Time Series"][dates]["4. close"])
        if dates not in VBMFX_dates:
            VBMFX_dates.append(dates)


VBMFX_data = pd.DataFrame({"Dates": VBMFX_dates, "VBMFX_Close": VBMFX_close})
VBMFX_data

,Dates,VBMFX_Close
0,2020-01-03,11.1200
1,2019-12-31,11.0500
2,2019-11-29,11.0900
3,2019-10-31,11.1200
4,2019-09-30,11.1200
...,...,...
235,2000-06-30,9.6100
236,2000-05-31,9.4600
237,2000-04-28,9.5300
238,2000-03-31,9.6300


In [17]:
# Merge the two SPY and VBMFX dataframes
data = SPY_data.merge(VBMFX_data, on="Dates")

# Change all column dtypes to datetime or float as appropriate
data["Dates"] = pd.to_datetime(data["Dates"])
data["SPY_Close"] = pd.to_numeric(data["SPY_Close"])
data["VBMFX_Close"] = pd.to_numeric(data["VBMFX_Close"])

data = data.set_index('Dates')

data.dtypes

SPY_Close      float64
VBMFX_Close    float64
dtype: object

In [18]:
data

,SPY_Close,VBMFX_Close
Dates,,
2020-01-03,322.4100,11.12
2019-12-31,321.8600,11.05
2019-11-29,314.3100,11.09
2019-10-31,303.3300,11.12
2019-09-30,296.7700,11.12
...,...,...
2000-06-30,145.2812,9.61
2000-05-31,142.8125,9.46
2000-04-28,145.0937,9.53


In [22]:
# Calculate annual percentage change

data_12M_Return = data.pct_change(12)
data_12M_Return = data_12M_Return.dropna()

data_12M_Return

,SPY_Close,VBMFX_Close
Dates,,
2019-01-31,-0.162774,-0.053058
2018-12-31,-0.223513,-0.054299
2018-11-30,-0.123000,-0.072137
2018-10-31,-0.107803,-0.077338
2018-09-28,-0.020386,-0.068345
...,...,...
2000-06-30,0.185002,-0.042829
2000-05-31,0.133433,-0.057769
2000-04-28,0.161679,-0.048902
